In [155]:
import pandas as pd
import json
import re
import numpy as np
import ast
from pathlib import Path  

In [156]:
main_path = "/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/"

In [157]:
# Define the data types
data_types = ['string', 'array', 'number', 'boolean', 'object']
columns = ['col_path_1','col_path_2','col_path_3','col_path_4', 'col_path_5']


def process_row_path(row, columns, data_types):
    row['data_type'] = ''
    for column in columns:
        if row[column] in data_types:
            row['data_type'] = row[column]
            row[column] = np.nan
        else:
            row[column]
    return row


In [158]:
def file_paths(df):
    # Create different colums for each part of the document path
    df['path_1'] = df['variable'].str.split('/', n=1).str[0]
    df['path_2'] = df['variable'].str.split('/', n=3).str[1]
    df['path_3'] = df['variable'].str.split('/', n=3).str[2]
    df['path_4'] = df['variable'].str.split('/', n=3).str[3]

    # Create a column with the JSON name
    df['json_name'] = df['variable'].str.rsplit('/', n=1).str[-1]


    # As the JSON name is stored in the json_name column, fill other parts of the path with Na if the name of the JSON is present
    mark = ".json"

    df['path_2'] = df['path_2'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)
    df['path_3'] = df['path_3'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)
    df['path_4'] = df['path_4'].apply(lambda x: np.nan if isinstance(x, str) and mark in x else x)

    # Unlist the value column (where the JSON info is stored)
    for index, row in df.iterrows():
        if isinstance(row['value'], list):
            df.at[index, 'value'] = row['value'][0]
    
    return df

In [159]:
def string_to_dict(s):
    if ',' not in s and ':' not in s:
        return s
    result = {}
    items = s.split(',')
    for item in items:
        if ':' not in item:
            continue  
        key, value = item.split(':', 1)  # use maxsplit=1 to avoid unpacking issues
        try:
            key = eval(key.strip())
            value = eval(value.strip())
        except Exception as e:
            continue
        result[key] = value
    return result

    

In [160]:
"""
def row_column_paths(df):
# Initialize the new columns
    df['row_path'] = '' 
    df['col__path_1'] = ''  
    df['col__path_2'] = ''  
    df['col__path_3'] = ''
    df['col__path_4'] = ''
    df['col__path_5'] = '' 

    # Extract the level 1 keys 
    for index, row in df.iterrows():
        df.at[index, 'row_path'] = list(row['value'].keys())
        
    df['row_path'] = df['row_path'].astype('str').apply(eval).str[0]

    # Take the level 1 keys stored in a list and store them in individual rows
    df = df.explode('row_path')


    # For the level 1 keys stored in col_path, extract the level 2 keys and store in row_path_1 
    df['col_path_1_values'] = df.apply(lambda row: [row['value'].get(row['row_path'], None)], axis=1)
    


    # Unlist and store in individual rows
    df['col_path_1_values'] = df['col_path_1_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['col_path_1_values'] = df['col_path_1_values'].apply(lambda x: x[0] if isinstance(x, list) else x)


    

    # Assign row_path_1_values to row_path_1
    df['col_path_1'] = df['col_path_1_values']
    df = df.explode('col_path_1')


    df['col_path_1_values'] = df['col_path_1_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)

    # For the level 2 keys stored in row_path_1, extract the level 3 keys and store in row_path_2 
    #df['row_path_2_values'] = df.apply(lambda row: [row['row_path_1_values'].get(row['row_path_1'], None)] if isinstance(row['row_path_1_values'], dict) else None, axis=1)
    df['col_path_2_values'] = df.apply(lambda row: [row['col_path_1_values'].get(row['col_path_1'], None)]if isinstance (row['col_path_1_values'], dict) else None, axis=1)
    
    
    # Unlist and store in individual rows
    df['col_path_2_values'] = df['col_path_2_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['col_path_2_values'] = df['col_path_2_values'].apply(lambda x: x[0] if isinstance(x, list) else x)

    

    # Assign row_path_2_values to row_path_2
    df['col_path_2'] = df['col_path_2_values']
    df = df.explode('col_path_2')

    df['col_path_2_values'] = df['col_path_2_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    
    

    #ROW_PATH_3
    # For the level 3 keys stored in row_path_2, extract the level 4 keys and store in row_path_3 (previously json_4)
    df['col_path_3_values'] = df.apply(lambda row: [row['col_path_2_values'].get(row['col_path_2'], None)] if isinstance (row['col_path_2_values'], dict) else None, axis=1)

    # Unlist and store in individual rows
    df['col_path_3_values'] = df['col_path_3_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['col_path_3_values'] = df['col_path_3_values'].apply(lambda x: x[0] if isinstance(x, list) else x)


    df['col_path_3_values'] = df['col_path_3_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    # Assign row_path_3_values to row_path_3
    df['col_path_3'] = df['col_path_3_values']
    df = df.explode('col_path_3')



    
    #ROW_PATH_4
    # For the level 3 keys stored in row_path_2, extract the level 4 keys and store in row_path_3 (previously json_4)
    df['col_path_4_values'] = df.apply(lambda row: [row['col_path_3_values'].get(row['col_path_3'], None)]if isinstance (row['col_path_3_values'], dict) else None, axis=1)
    
    df['col_path_4_values'] = df['col_path_4_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['col_path_4_values'] = df['col_path_4_values'].apply(lambda x: x[0] if isinstance(x, list) else x)

    display(df[['col_path_3', 'col_path_4_values']])
    
    df['col_path_4_values'] = df['col_path_4_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    # Assign row_path_3_values to row_path_3
    df['col_path_4'] = df['col_path_4_values']
    df = df.explode('col_path_4')


    #ROW_PATH_5
    # For the level 3 keys stored in row_path_2, extract the level 4 keys and store in row_path_3 (previously json_4)
    df['col_path_5_values'] = df.apply(lambda row: [row['col_path_4_values'].get(row['col_path_4'], None)]if isinstance (row['col_path_4_values'], dict) else None, axis=1)
    
    df['col_path_5_values'] = df['col_path_5_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['col_path_5_values'] = df['col_path_5_values'].apply(lambda x: x[0] if isinstance(x, list) else x)

    display(df[['col_path_4', 'col_path_5_values']])
    
    df['col_path_5_values'] = df['col_path_5_values'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    
    # Assign row_path_3_values to row_path_3
    df['col_path_5'] = df['col_path_5_values']
    df = df.explode('col_path_5')
    
   

    return df
"""

"\ndef row_column_paths(df):\n# Initialize the new columns\n    df['row_path'] = '' \n    df['col__path_1'] = ''  \n    df['col__path_2'] = ''  \n    df['col__path_3'] = ''\n    df['col__path_4'] = ''\n    df['col__path_5'] = '' \n\n    # Extract the level 1 keys \n    for index, row in df.iterrows():\n        df.at[index, 'row_path'] = list(row['value'].keys())\n\n    df['row_path'] = df['row_path'].astype('str').apply(eval).str[0]\n\n    # Take the level 1 keys stored in a list and store them in individual rows\n    df = df.explode('row_path')\n\n\n    # For the level 1 keys stored in col_path, extract the level 2 keys and store in row_path_1 \n    df['col_path_1_values'] = df.apply(lambda row: [row['value'].get(row['row_path'], None)], axis=1)\n\n\n\n    # Unlist and store in individual rows\n    df['col_path_1_values'] = df['col_path_1_values'].apply(lambda x: x[0] if isinstance(x, list) else x)\n    df['col_path_1_values'] = df['col_path_1_values'].apply(lambda x: x[0] if isinstan

In [161]:
def row_column_paths(df):
    # Initialize the new columns
    df['row_path'] = '' 
    df['col_path_1'] = ''  
    df['col_path_2'] = ''  
    df['col_path_3'] = ''
    df['col_path_4'] = ''
    df['col_path_5'] = '' 

    # Extract the level 1 keys 
    for index, row in df.iterrows():
        df.at[index, 'row_path'] = list(row['value'].keys())

    df['row_path'] = df['row_path'].astype('str').apply(eval).str[0]

    # Take the level 1 keys stored in a list and store them in individual rows
    df = df.explode('row_path')

    """
    df['value'] = df['value'].apply(
        lambda x: string_to_dict(x) if isinstance(x, str) else x)
    
    # For the level 1 keys stored in col_path, extract the level 2 keys and store in row_path_1 
    df['col_path_1_values'] = df.apply(lambda row: [row['value'].get(row['row_path'], None)], axis=1)
    
    # Unlist and store in individual rows
    df['col_path_1_values'] = df['col_path_1_values'].apply(lambda x: x[0] if isinstance(x, list) else x)
    df['col_path_1_values'] = df['col_path_1_values'].apply(lambda x: x[0] if isinstance(x, list) else x)


    # Assign row_path_1_values to row_path_1
    df['col_path_1'] = df['col_path_1_values']
    df = df.explode('col_path_1')
    """

    def column_paths(df, row_var, get_var, unlist_var, explode_var):

        print("before", df[row_var].apply(type).value_counts())
        
        # Convert to dict if it's a string
        df[row_var] = df[row_var].apply(
            lambda x: string_to_dict(x) if isinstance(x, str) else x)
        
        print("After", df[row_var].apply(type).value_counts())
        
        # Extract nested values using keys
        df[unlist_var] = df.apply(lambda row: [row[row_var].get(row[get_var], None)] if isinstance (row[row_var], dict) else None, axis=1)


        # Unlist and explode
        df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)
        df[unlist_var] = df[unlist_var].apply(lambda x: x[0] if isinstance(x, list) else x)
        
        df[explode_var] = df[unlist_var]
        df = df.explode(explode_var)

    

        return df

    row_var = ['value', 'col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values']
    get_var = ['row_path', 'col_path_1', 'col_path_2', 'col_path_3', 'col_path_4']
    unlist_var = ['col_path_1_values', 'col_path_2_values', 'col_path_3_values', 'col_path_4_values', 'col_path_5_values']
    explode_var = ['col_path_1', 'col_path_2', 'col_path_3', 'col_path_4', 'col_path_5']

    for r, g, u, e in zip(row_var, get_var, unlist_var, explode_var):
        df = column_paths(df, r, g, u, e)
        row_var = unlist_var

    return df


In [162]:
def clean_and_store(df, file_name):
 
    # Reorder the columns in the df
    df = df.loc[:, ['variable', 'value', 'path_1', 'path_2',
        'path_3', 'path_4', 'json_name', 'row_path', 'col_path_1','col_path_2','col_path_3', 'col_path_4', 'col_path_5','data_type', 'col_path_1_values','col_path_2_values','col_path_3_values', 'col_path_4_values', 'col_path_5_values']]
    
    df.reset_index(drop=True, inplace=True)
    df = df.fillna('Missing')
    #df = df.astype(str)
    


    df.to_csv(f"{main_path}Instagram/Output/Output_" + file_name + '.csv', index=False)

    return df

In [163]:
def structure_donations(data):

    data = Path(data)  
    #file_name = data.name  # Extracts "data.json"
    file_name = Path(data).stem 

    # Load JSON file
    with open(data, 'r') as f:
        data = json.load(f)

    # Flatten JSON (handling nested structures)
    df = pd.json_normalize(data, max_level=0)

    # Delete user specific informations
    df.columns = df.columns.str.replace(r'^[^/]+/', '', regex=True)

    # Extract column names
    cols = df.columns.tolist()


    # From wide to long df
    df = pd.melt(df, value_vars= cols)

    df = file_paths(df)
    
    df = row_column_paths(df)
    
    df = df.apply(lambda row: process_row_path(row, columns, data_types), axis=1)
    
    df = clean_and_store(df, file_name)
    
    
    return df


In [164]:
input_directory = Path(f'{main_path}Instagram/Input')  
print(input_directory)

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Instagram/Input


In [165]:

for file in input_directory.iterdir():  
    if file.is_file():  
        print(file)
        structure_donations(file)
        

/home/rvissche/Nextcloud/What-If/what-if-data-donation/what-if-data-donation/json_structure_donations/processed_json_structure_donations/Instagram/Input/08-04-2025-json_structure instagram.json
before value
<class 'dict'>    35
Name: count, dtype: int64
After value
<class 'dict'>    35
Name: count, dtype: int64
before col_path_1_values
<class 'dict'>    70
<class 'str'>      9
Name: count, dtype: int64
After col_path_1_values
<class 'dict'>    70
<class 'str'>      9
Name: count, dtype: int64
before col_path_2_values
<class 'dict'>        90
<class 'str'>         30
<class 'NoneType'>     9
Name: count, dtype: int64
After col_path_2_values
<class 'dict'>        90
<class 'str'>         30
<class 'NoneType'>     9
Name: count, dtype: int64
before col_path_3_values
<class 'dict'>        160
<class 'NoneType'>     56
<class 'str'>          15
Name: count, dtype: int64
After col_path_3_values
<class 'dict'>        160
<class 'NoneType'>     56
<class 'str'>          15
Name: count, dtype: 

In [166]:
# Path to the folder containing CSV files
output_path = f"{main_path}Instagram/Output"

# Get a list of all CSV files in the folder
csv_files = list(Path(output_path).glob("*.csv"))

# Load all CSVs into a list of DataFrames
dfs = [pd.read_csv(file) for file in csv_files]


common_columns = ['variable', 'value', 'path_1', 'path_2',
        'path_3', 'path_4', 'json_name', 'row_path', 'col_path_1','col_path_2','col_path_3', 'col_path_4', 'col_path_5','data_type', 'col_path_1_values','col_path_2_values','col_path_3_values', 'col_path_4_values', 'col_path_5_values']


merged_df = dfs[0]  # Start with the first DataFrame
for df in dfs[1:]:  # Merge with the rest  #
    merged_df = merged_df.merge(df, on=common_columns, how="outer")




# Filter where col1 contains 'messages', then drop duplicates based on col2
df_filtered = merged_df[merged_df["path_1"] == "messages"].drop_duplicates(subset="path_2")


# Append rows where col1 does not contain 'messages'
df_final = pd.concat([df_filtered, merged_df[merged_df["path_1"] != "messages"]], ignore_index=True)
df_final = df_final.replace('Missing', np.nan)


print(df_final.isna().sum())

# Save the final merged DataFrame
df_final.to_csv(f"{main_path}Instagram/Final/Merged_structures_IG.csv", index=False)

variable                0
value                   0
path_1                  0
path_2                967
path_3               1351
path_4               1415
json_name               0
row_path                0
col_path_1            130
col_path_2            594
col_path_3            763
col_path_4           1370
col_path_5           1406
data_type              61
col_path_1_values       0
col_path_2_values     130
col_path_3_values     594
col_path_4_values     763
col_path_5_values    1370
dtype: int64


/tmp/ipykernel_157938/1853675807.py:28: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final = df_final.replace('Missing', np.nan)
